# Normalization pipeline

Die Quelldaten sind aus meinem ersten RAG-Projekt und wurden mit fruendlicher Genehmigung von einer Webseite gescrappt.

**Was zu tun ist:**
- Daten von Marketingaussagen und Firmengeschichte befreien
- Die Specs normalisieren

Das meiste werde ich mit einem LLM machen. Die Ausgangsdaten werden stichprobenartig kontrolliert. Ziel ist ein funktionierendes und korrektes retrival, welches nicht zwingend mit der realen Welt (korrekte technische Daten z.B.) übereinstimmen muss. In einem Realworldprojekt ist das selbstverständlich etwas anderes.

Wärend der Bereinigung werden die Daten in Specs und Descriptions aufgeteilt.

In [28]:
import os
import json
import time
import pandas as pd
from tqdm import tqdm
from mistralai import Mistral
from dotenv import load_dotenv

load_dotenv()

df = pd.read_json('./products_raw.json')

api_key = os.getenv('MISTRAL_API_KEY')
model = 'mistral-medium-2508'
client = Mistral(api_key=api_key)

descs_promt = open('./descs_agent.md', 'r').read()
specs_promt = open('./specs_agent.md', 'r').read()
summs_promt = open('./summs_agent.md').read()

descs_chunks = []
specs_chunks = []
summs_chunks = []

Die Beschreibungen werden über ein LLM bereinigt. Der Systempromt liegt als md-File vor. 

### Funktionen

- **agent_request**: Allgemeine Retrivalfunktion. Fallbackfunktion (wegen fehlenden Specs integriert)
- **format_spec**: Bereitet Embeddingdocs für Specs auf

In [29]:
def agent_request(promt, content, fallback):

    if content == '[]': content = fallback
    
    response = client.chat.complete(
        model = model,
        messages = [
            {
                'role': 'system',
                'content': promt
            },
            {
                'role': 'user',
                'content': content,
            }
        ],
        response_format = {"type": "json_object"}
    )

    return response

def format_spec(group_name, specs_list):
    if not specs_list:
        return ''
    
    if '-' in group_name:
        label, unit = group_name.rsplit('-', 1)
        header = f"{label} ({unit})"
    else:
        header = group_name
    
    spec_parts = []

    for key, value in specs_list.items():
        readable_key = key.replace('_', ' ')
        spec_parts.append(f"{readable_key}: {value}")

    return f"{header} - {', '.join(spec_parts)}"

# Erstellung der Chunks

Die Daten werden an ein LLM geliefert und wir erhalten nur die erzeugen Artefarkte. Diese werden in des Schema der Chunks integriert:

In [ ]:
for index, row in tqdm(df.iterrows(), total=len(df)):

    summs_response = agent_request(summs_promt, json.dumps(row.to_dict(), ensure_ascii=False), False)
    descs_response = agent_request(descs_promt, row['description'], False)
    specs_response = agent_request(specs_promt, json.dumps(row['specs'], ensure_ascii=False), row['description'])

    summs_result = json.loads(summs_response.choices[0].message.content)                              
    descs_result = json.loads(descs_response.choices[0].message.content)
    specs_result = json.loads(specs_response.choices[0].message.content)

    print(summs_result)
    print(descs_result)
    print(specs_result)

    category = summs_result['category']
    manufacturer = summs_result['manufacturer']
    title = summs_result['title']

    product_metadata = {
        'product_id': row['id'],
        'product_title': title,
        'product_url': row['url'],
        'product_category': category,
        'product_manufacturer': manufacturer,
        'product_price': row['price'],
        'product_sku': row['order_number']
    }

    summs_chunks.append({
        'document': summs_result['summary'],
        'metadata': {
            'chunk_id': f"{row['id']}_summ_0",
            'chunk_type': 'description',
            **product_metadata
        }
    })

    for i, pararagraph in enumerate(descs_result):
        descs_chunks.append({
            'document': pararagraph,
            'metadata': {
                'chunk_id': f"{row['id']}_desc_{i}",
                'chunk_type': 'description',
                **product_metadata
            }
        })

    for i, (group_name, specs_list) in enumerate(specs_result.items()):
        if specs_list:
            specs_chunks.append({
                'document': format_spec(group_name, specs_list),
                'metadata': {
                    'chunk_id': f"{row['id']}_spec_{i}",
                    'chunk_type': 'specs',
                    **product_metadata
                }
            })

    # Testing only
    # if index == 65: break

summs_df = pd.DataFrame(summs_chunks)
descs_df = pd.DataFrame(descs_chunks)
specs_df = pd.DataFrame(specs_chunks)

  0%|          | 0/151 [00:00<?, ?it/s]

  1%|          | 1/151 [00:19<48:30, 19.40s/it]

{'summary': 'Der Kirsch LABO-288 PRO-ACTIVE ist ein Laborkühlschrank mit einem Nutzvolumen von 280 Litern und einer Temperaturregelung von ca. 0 bis +15 °C. Der Kühlschrank verfügt über eine statisch belüftete, geräuscharme und energiesparende Kältemaschine, die für 220-240 V Wechselstrom ausgelegt ist. Die automatische Abtauung und die 55 mm starke Isolierung sorgen für eine effiziente Kühlleistung. Der Innenraum besteht aus glattem Aluminium mit einer farblosen Schutzbeschichtung und ist mit drei kunststoffbeschichteten Rosten ausgestattet. Der Laborkühlschrank ist mit einer elektronischen Temperatursteuerung ausgestattet, die eine hohe Temperaturkonstanz bei geringem Energieverbrauch gewährleistet. Zusätzlich verfügt er über ein akustisches und optisches Alarmsignal bei abweichenden Temperaturen, eine Sicherheitseinrichtung gegen Minus-Temperaturen und einen potentialfreien Kontakt zum Anschluss an die zentrale Leittechnik. Die Umluftkühlung mit einem Querstromgebläse sorgt für eine

  1%|▏         | 2/151 [00:44<56:08, 22.61s/it]

{'summary': 'Der Kirsch FROSTER LABEX-530 ULTIMATE ist ein Laborgefrierschrank, speziell für die Lagerung von leicht entzündlichen und explosiven Stoffen konzipiert. Mit einem explosionsgeschützten Innenraum, der die Sicherheitsanforderungen der Zone 2 erfüllt und durch ein TÜV-Zertifikat zertifiziert ist, bietet er höchste Sicherheit. Der Gefrierschrank verfügt über eine geräuscharme und hermetisch erschütterungsfreie Kältemaschine mit automatischer Abtauung und einer Isolierung von 70 mm Stärke. Die Temperaturregelung reicht von -10°C bis -30°C und wird durch ein Umluftgekühltes System mit Querstromgebläse unterstützt, das bei geöffneten Türen automatisch abschaltet. Der Innenraum besteht aus poliertem Aluminium mit einer farblosen Schutzbeschichtung und bietet 500 Liter Kühlinhalt. Der Kirsch FROSTER LABEX-530 ULTIMATE ist mit einem Touch-Display, Zugangskontrolle mit Protokoll und grafischer Anzeige des Temperaturverlaufs ausgestattet. Zudem verfügt er über eine USB-Schnittstelle z

...

{'summary': 'Der Liebherr LCexv 4010 MediLine ist ein hochwertiger Laborkühlschrank mit integriertem Gefrierteil, der speziell für die Anforderungen von Laboratorien entwickelt wurde. Mit einem Kühlteil von 240 Litern und einem Gefrierteil von 105 Litern bietet er ausreichend Platz für die Lagerung von temperaturempfindlichen Substanzen. Der Kühlteil arbeitet im Temperaturbereich von +3°C bis +16°C, während das Gefrierteil Temperaturen von -9°C bis -30°C erreicht. Der LCexv 4010 verfügt über eine präzise Komfortelektronik mit 1/10°-Temperaturanzeige und ist mit einem Sicherheitsthermostat ausgestattet, das im Fehlerfall verhindert, dass die Innentemperatur unter +2°C fällt. Zudem erfüllt er die NF X 15-140 Norm hinsichtlich maximaler Temperaturstabilität und optimaler Temperaturkonstanz. Der Kühlschrank ist mit einem optischen und akustischen Alarmsystem ausgestattet, das bei Temperaturabweichungen und zu lange geöffneter Tür warnt. Ein integrierter Datenspeicher protokolliert kontinui

 43%|████▎     | 65/151 [20:46<36:16, 25.30s/it]

{'summary': 'Der Kirsch LABEX-720 ULTIMATE ist ein Laborkühlschrank mit explosionsgeschütztem Innenraum, speziell für die Lagerung von leicht entzündlichen und explosiven Stoffen. Mit einem Nutzvolumen von 700 Litern und einer Temperaturregelung von +0°C bis +15°C bietet er höchste Sicherheit und Temperaturkonstanz. Der Kühlschrank verfügt über eine PRO-ACTIVE-Steuerung, automatische Abtauung und eine Umluftkühlung mit Querstromgebläse. Zusätzlich ist er mit einem Touch-Display, Zugangskontrolle, grafischer Temperaturverlaufsanzeige und digitaler Datendokumentation über USB-Schnittstelle ausgestattet. Der Innenraum besteht aus poliertem Aluminium mit Schutzbeschichtung und sechs kunststoffbeschichteten Rosten. Das Außengehäuse ist aus verzinktem Stahlblech mit einer kratzfesten Pulverbeschichtung. Der Kirsch LABEX-720 ULTIMATE ist ideal für Laboratorien, Kliniken und das Gesundheitswesen.', 'category': 'Laborkühlschrank', 'manufacturer': 'Kirsch', 'title': 'LABEX-720 ULTIMATE'}
['Der K

 43%|████▎     | 65/151 [21:02<27:50, 19.42s/it]

{'summary': 'Der Kirsch MED-340 PRO-ACTIVE ist ein Medikamentenkühlschrank nach DIN 58345 mit einem Bruttofassungsvermögen von 330 Litern. Er verfügt über neun Schubfächer auf Rollen, die eine hohe Stauraumkapazität bieten, sowie über eine Temperaturregelung von +2°C bis +15°C. Der Kühlschrank ist mit einer automatischen Abtauung und einer Umluftkühlung ausgestattet, die eine gleichmäßige Temperaturverteilung gewährleistet. Der Innenbehälter besteht aus glattem Aluminium, und das Gerät zeichnet sich durch einen geringen Energieverbrauch von 0,82 kWh/24 h aus. Der MED-340 ist ideal für die sichere Lagerung von Medikamenten in medizinischen Einrichtungen.', 'category': 'Medikamentenkühlschrank', 'manufacturer': 'Kirsch', 'title': 'MED-340 PRO-ACTIVE'}
['Der Kirsch MED-340 ist ein nach DIN 58345 genormter Medikamentenkühlschrank mit PRO-Active-Steuerung. Er bietet eine hohe Staukapazität durch neun, auf Rollen gelagerten und arretierenden Schubfächern. Zur besseren Übersicht sind drei Län

## Quality Checks

Wir müssen zuletzt noch in die Chunks schauen ob sie die wichtigsten Rahmenbedinungen einhalten. Vor allem die Länge da die meisten Models eine beschränkte Anzahl an Tokes verarbeiten kann und die darf von den Chunks nicht überschritten werden.

In [31]:
def stats(df, treshhold, name):

    df['length'] = df['document'].str.len()
    small = df[df['length'] < treshhold]

    print(f'=== {name} ===')
    print(f"{df['length'].describe()}")
    print(f'TOTAL CHUNKS: {len(df)}')
    print(f'SHORT CHUNKS: {len(small)}')

    for idx, row in small.iterrows():
        
        print(f"{row['metadata']['product_title']}")
        print(f"Document: {row['document']}")
        print(f"Length: {row['length']}")
        print(f'--- --- ---')

    print('\n')


In [32]:
stats(summs_df, 200, 'Summs')
stats(descs_df, 200, 'Descriptions')
stats(specs_df, 100, 'Specs')

=== Summs ===
count      66.000000
mean      937.681818
std       190.068745
min       583.000000
25%       842.500000
50%       929.500000
75%      1036.500000
max      1605.000000
Name: length, dtype: float64
TOTAL CHUNKS: 66
SHORT CHUNKS: 0


=== Descriptions ===
count     394.000000
mean      350.614213
std       117.198591
min       116.000000
25%       280.000000
50%       330.000000
75%       403.500000
max      1336.000000
Name: length, dtype: float64
TOTAL CHUNKS: 394
SHORT CHUNKS: 16
HMFvh 5511 H63 Perfection
Document: Der HMFvh 5501-20 H63 ist ultra-effizient im Energieverbrauch und erfüllt die Sicherheitsstandards nach IEC 61010-2-011.
Length: 120
--- --- ---
Tiefkühltruhe TT 50-90
Document: Mit einem Nutzinhalt von 90 Litern und einem Innenraum frei von Anbauten bietet die TT 50-90 viel Platz für Ihre Proben. Die Temperaturregelung ist mit einer Genauigkeit von +/- 1K sehr präzise.
Length: 194
--- --- ---
Tiefkühltruhe TT 50-90
Document: Die Abmessungen der Tiefkühltruhe b

An dieser Stelle wurden weitere Verbesserungen an den Promts vorgenommen und getestet... Mit den Specs muss noch eine andere Strategie ausgedacht werden. Jetzt erstmal mit den kleinen Chunks probieren. 

## Speichern

In [33]:
summs_df.to_json('summs_chunks.jsonl', orient='records', lines=True, force_ascii=False)
descs_df.to_json('descs_chunks.jsonl', orient='records', lines=True, force_ascii=False)
specs_df.to_json('specs_chunks.jsonl', orient='records', lines=True, force_ascii=False)